Example
=======

As a first example, we use an image dataset and a pre-trained model to classify the images. We use the `ReVel` framework to load the dataset and the model, and to perform the classification. We also made use of the `procedures` module to help us with the classification process.

In [5]:
from SHIELD import SHIELD
from SHIELD.procedures import procedures
from ReVel.load_data import load_data
from ReVel.perturbations import get_perturbation

# Load the model
# Download the dataset Flowers and change the last layer to fit the number of classes
classifier = procedures.classifier("efficientnet-b2", num_classes=102)
perturbation = get_perturbation(name="square",dim=9,num_classes= 102,
    final_size=(224, 224),kernel=150.0,max_dist=20,ratio=0.5)

dataset = load_data("Flowers", perturbation=perturbation, train=True, dir="./data/")
classifier = classifier.train()

100%|██████████| 344862509/344862509 [00:05<00:00, 60466664.01it/s]


Extracting data/Flowers/train/flowers-102/102flowers.tgz to data/Flowers/train/flowers-102


100%|██████████| 502/502 [00:00<00:00, 3180574.94it/s]


100%|██████████| 14989/14989 [00:00<00:00, 84274025.01it/s]


100%|██████████| 344862509/344862509 [00:06<00:00, 54044619.73it/s]


Extracting data/Flowers/val/flowers-102/102flowers.tgz to data/Flowers/val/flowers-102


100%|██████████| 502/502 [00:00<00:00, 3185386.70it/s]


100%|██████████| 14989/14989 [00:00<00:00, 72179589.73it/s]
